### Import the Necessary Libraries

In [1]:
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

pd.options.display.max_columns = 999

/Users/mahsumkocabey/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/mahsumkocabey/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-1-03fae1edc20b>:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

### Read the Data

In [2]:
# Load the GeoJSON file
boroughs_gdf = gpd.read_file('../data/LAS/london_sport.json')
boroughs_gdf['name'] = boroughs_gdf['name'].str.title()

# Load the LAS Locations
las_data = pd.read_csv('../data/LAS/LAS Locations.csv')
# Load the LFB Locations
lfb_data = pd.read_csv('../data/LFB/LFB Locations.csv')

### Convert Them into Geographic Data

In [3]:
# Convert the DataFrame into a GeoDataFrame
geometry_lfb = [Point(xy) for xy in zip(lfb_data.Longitude, lfb_data.Latitude)]
lfb_gdf = gpd.GeoDataFrame(lfb_data, geometry=geometry_lfb)

# Set the original CRS to WGS 84
lfb_gdf.set_crs(epsg=4326, inplace=True)

# Convert the CRS to UK National Grid
lfb_gdf = lfb_gdf.to_crs(epsg=27700)

In [4]:

# Convert the DataFrame into a GeoDataFrame
geometry = [Point(xy) for xy in zip(las_data.Longitude, las_data.Latitude)]
las_gdf = gpd.GeoDataFrame(las_data, geometry=geometry)

# Set the original CRS to WGS 84
las_gdf.set_crs(epsg=4326, inplace=True)

# Convert the CRS to UK National Grid
las_gdf = las_gdf.to_crs(epsg=27700)

### Create a Map with Station Locations Only

In [5]:
# Create a map centered around London
lfb_m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Add a marker for each location in the DataFrame
for index, row in lfb_data.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], 
                  popup=f"{row['Station/code']}, {row['Address']}"
                 ).add_to(lfb_m)

# Add the GeoJSON to the folium map
folium.GeoJson(
    boroughs_gdf,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': 'yellow',  # you can also use different colors, it's up to you
        'color': 'red',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.3,
    }
).add_to(lfb_m)

folium.TileLayer('cartodbpositron').add_to(lfb_m)

lfb_m.save('lfb_map.html')

# Show the map
lfb_m

In [6]:

# Create a map centered around London
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)

# Add a marker for each location in the DataFrame
for index, row in las_data.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], 
                  popup=f"{row['Station/code']}, {row['Address']}"
                 ).add_to(m)

# Add the GeoJSON to the folium map
folium.GeoJson(
    boroughs_gdf,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': 'yellow',  # you can also use different colors, it's up to you
        'color': 'red',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.3,
    }
).add_to(m)

folium.TileLayer('cartodbpositron').add_to(m)

m.save('map.html')

# Show the map
m

------------

### Read LAS and LFB Calls Data

In [7]:
# Read the LAS Calls DATA

# LAS Calls
file_path = '../data/LAS/DataApril2023/LAS-From 2018 to 2023.xlsx'
IncidentsLSOA = pd.read_excel(file_path, engine='openpyxl', sheet_name='IncidentsLSOA')
# LFB Calls
london = pd.read_csv('../data/LFB/LFB Incident data 2020 to 2023.csv')

# Read the .xls file
LSOA_country_of_birth = pd.read_excel('../data/LSOA-Census-Demographics/Country of birth.xlsx',
                                      engine='openpyxl', sheet_name='2021')
# Latitude and Longitude of Borough Data
lsoa_location = pd.read_csv("../data/LSOA-Census-Demographics/lsoa_latlong.csv")

**Filter it for the last 2 years**

**LAS**

In [8]:
# Create a mask of boolean values indicating whether each date is within the specified range
mask_two = IncidentsLSOA['year'] >= 2021

# Apply the mask to the DataFrame to get a new DataFrame with dates within the specified range
IncidentsLSOA = IncidentsLSOA.loc[mask_two].reset_index(drop=True)

# merge with inner
merged_df = IncidentsLSOA.merge(LSOA_country_of_birth, how='inner', left_on='LSOA11CD', right_on='LSOA code')
# drop the 'LSOA code' column
merged_df = merged_df.drop(columns=['LSOA code'])

# merge with inner
merged_df = merged_df.merge(lsoa_location, how='inner', left_on='LSOA11CD', right_on='lsoa11cd')
# drop the 'LSOA code' column
merged_df = merged_df.drop(columns=['lsoa11cd'])

las_by_borough_last_2_years = merged_df.groupby(["local authority name"])["total_incidents"].\
                                                                        sum().sort_values(ascending=False)

# Convert your series to a DataFrame
las_df = las_by_borough_last_2_years.reset_index()
las_df.columns = ['name', 'total_incidents']
las_df['name'] = las_df['name'].str.title()

**LFB**

In [9]:
london['DateOfCall'] = pd.to_datetime(london['DateOfCall']).dt.date

start_date = pd.Timestamp("2021-01-01").date()
end_date = pd.Timestamp("2023-01-31").date()

# Filter dataframe based on date range
london_filtered = london[(london['DateOfCall'] >= start_date) & (london['DateOfCall'] <= end_date)]

lfb_by_borough = london_filtered.groupby(['ProperCase'])["NumCalls"].sum().sort_values(ascending=False)

# Convert series to dataframe
lfb_by_borough_df = lfb_by_borough.reset_index()
lfb_by_borough_df.columns = ['name', 'total_incidents']
lfb_by_borough_df['name'] = lfb_by_borough_df['name'].str.title()

## Choropleth Map: Total Incidents per boroughs vs LAS Locations

In [10]:
# Create a map centered around London with cartodbpositron tiles
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10, tiles='cartodbpositron')

# Add a marker for each location in the DataFrame
for index, row in las_data.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], 
                  popup=f"{row['Station/code']}, {row['Address']}"
                 ).add_to(m)

# Convert the borough boundaries to json format
geo_json_data = boroughs_gdf.to_json()

# Create the choropleth map. Key_on refers to the borough name in the GeoJSON file
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=las_df,
    columns=['name', 'total_incidents'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='black',
    legend_name='Total Incidents'
).add_to(m)

folium.LayerControl().add_to(m)

m.save('choropleth_map_v1.html')

# Show the map
m

## Choropleth Map: Total Incidents per boroughs vs LFB Locations

In [12]:
# Create a map centered around London with cartodbpositron tiles
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10, tiles='cartodbpositron')

# Add a marker for each location in the DataFrame
for index, row in lfb_data.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], 
                  popup=f"{row['Station/code']}, {row['Address']}"
                 ).add_to(m)

# Convert the borough boundaries to json format
geo_json_data = boroughs_gdf.to_json()

# Create the choropleth map. Key_on refers to the borough name in the GeoJSON file
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=lfb_by_borough_df,
    columns=['name', 'total_incidents'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='blue',
    legend_name='Total Incidents'
).add_to(m)

folium.LayerControl().add_to(m)

m.save('lfb_choropleth_map_v1.html')

# Show the map
m